In [ ]:
t1="2018-08-29T00:00:00"
t2="2018-09-20T00:00:00"
nscw=5
chi2_limit=1.2
systematic_fraction=0.01

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt


import numpy as np
import importlib
from astroquery.simbad import Simbad
from astropy import coordinates as coord
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print("t1,t2",t1,t2)

In [ ]:
result_table = Simbad.query_object("Crab", wildcard=True)
crab_coord = coord.SkyCoord(result_table['RA'][0], result_table['DEC'][0], unit=("hourangle", "deg"))

In [ ]:
import requests

scw_picks={}

def scwlist(kind):
    print(t1,t2)
    r=requests.get("https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw/timesystem/api/v1.0/scwlist/{kind}/{t1}/{t2}".format(
                    kind=kind,
                    t1=t1,
                    t2=t2,
                ),
                  params=dict(
                      ra=crab_coord.ra.deg,
                      dec=crab_coord.dec.deg,
                      radius=10,
                  ))

    return [s for s in r.json() if s.endswith("0010")]

scwlist_cons = scwlist('cons')
scwlist_nrt = scwlist('nrt')


scw_pick = [
            s+"."+("001" if s in scwlist_cons else "000")
            for s in list(sorted(set( scwlist_nrt + scwlist_cons ))) 
            if s.endswith("0010")
        ][:nscw]


scw_list_str = ",".join(scw_pick)
scw_list_str

In [ ]:
import oda_api.api
import importlib
importlib.reload(oda_api)

disp = oda_api.api.DispatcherAPI(host="https://www.astro.unige.ch/cdci/astrooda/dispatch-data/")
#disp = oda_api.api.DispatcherAPI(host="http://cdcihn.isdc.unige.ch/staging-1.2/dispatcher")

#disp.get_instrument_description("isgri")



In [ ]:
crab_by_osa={}


#for osa_version in "OSA11.0", "OSA10.2":
for osa_version in ["OSA11.0"]:
    crab_by_lt={}
    crab_by_osa[osa_version]=crab_by_lt

    for c_emin in np.linspace(20,35,5):

        image = disp.get_product(instrument="isgri", 
                         product="isgri_image", 
                         product_type="Real", 
                         osa_version=osa_version,
                         E1_keV=np.round(c_emin),
                         E2_keV=80.0,
                         scw_list=scw_list_str)

        crab=image.dispatcher_catalog_1.table[np.argmax(image.dispatcher_catalog_1.table['significance'])]

        d=image.mosaic_image_0.data_unit[1].data


        img=np.array(d.data)

        m_bkg=img<10
        m_bkg&=img!=0

        img_std = np.std(img[m_bkg])

        img[np.array(img)>img_std*5]=img_std*5

        plt.imshow(img)

        crab_by_lt['%.10lg'%c_emin]=dict(
            emin=c_emin,            
            imgstd=img_std,
        )
        crab_by_lt['%.10lg'%c_emin].update(dict([(n, crab[n]) for n in crab.colnames]))


In [ ]:
for lt, c in crab_by_lt.items():
    print(lt, c['significance'],c['significance']/c['imgstd'])

In [ ]:
spectrum = disp.get_product(instrument="isgri", 
                 product="isgri_spectrum", 
                 product_type="Real", 
                 osa_version='OSA11.0',
                 E1_keV=25.0,
                 E2_keV=80.0,
                 scw_list=scw_list_str)

In [ ]:
crab_specprod=[l for l in spectrum._p_list if l.meta_data['src_name'] == "Crab"]    

crab_specprod[0].write_fits_file("/tmp/isgri_spectrum_Crab.fits")
crab_specprod[2].write_fits_file("/tmp/isgri_rmf_Crab.fits")

In [ ]:

from IPython.display import Image
from IPython.display import display 
import xspec
import shutil



In [ ]:
importlib.reload(xspec)

fit_by_lt = {}

fn_by_lt={}

xspec.AllModels.systematic=systematic_fraction   

for c_emin in np.linspace(25,40,15):
    xspec.AllData.clear()

    s = xspec.Spectrum("/tmp/isgri_spectrum_Crab.fits")   
    s.response="/tmp/isgri_rmf_Crab.fits"
    
    ig="**-%.2f 400.-**"%c_emin
    
    lt_key='%.10lg'%c_emin
    
    print(ig)
    s.ignore(ig)

      
    if False:
        m = xspec.Model("grbm")
        m.grbm.tem=600
        m.grbm.tem.frozen=True
        
        xspec.Fit.perform()

        print(m.grbm.alpha.values,m.grbm.beta.values, xspec.Fit.statistic/xspec.Fit.dof)
    else:
        m = xspec.Model("bknpo")
        m.bknpower.BreakE=100
        m.bknpower.BreakE.frozen=True
      
        xspec.Fit.perform()

        print(m.bknpower.PhoIndx1.values,m.bknpower.PhoIndx2.values, )
        
        fit_by_lt[lt_key]=dict(
            emin=c_emin,
            chi2_red=xspec.Fit.statistic/xspec.Fit.dof,
            PhoIndx1=m.bknpower.PhoIndx1.values,
            PhoIndx2=m.bknpower.PhoIndx2.values,
        )
        
    
    xspec.Plot.device="/png"
    #xspec.Plot.addCommand("setplot en")
    xspec.Plot.xAxis="keV"
    xspec.Plot("ldata del")
    xspec.Plot.device="/png"
    
    fn="fit_lt%.5lg.png"%c_emin
    
    fn_by_lt[lt_key] = fn
    
    shutil.move("pgplot.png_2", fn)

    _=display(Image(filename=fn,format="png"))



In [ ]:
fit_by_lt

In [ ]:

for lt,d in fit_by_lt.items():
    print(lt, d['chi2_red'])
    
good_lt = min([p for p in fit_by_lt.items() if p[1]['chi2_red']<chi2_limit], key=lambda x:x)

good_lt_next = min([p for p in fit_by_lt.items() if p[1]['chi2_red']<chi2_limit*1.2], key=lambda x:x)

good_lt[0],good_lt_next[0]

In [ ]:
fn_picked=fn_by_lt[good_lt[0]]
fn_next_picked=fn_by_lt[good_lt_next[0]]

In [ ]:
def run_local():
    pass

In [ ]:
def run_cdci():
    pass

In [ ]:
def run_ddosa():
    pass

In [ ]:
summary={
    'status': 'OK',
    'emin_good': good_lt,
}

In [ ]:
summary=summary
fit_results=fit_by_lt
crab_by_osa=crab_by_osa
good_fit_png=fn_picked
next_good_fit_png=fn_next_picked